In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import os

# ----------------------------
# 1. CNN Model Definition
# ----------------------------
class ChartClassifier(nn.Module):
    def __init__(self):
        super(ChartClassifier, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
        )

        self.fc_layers = nn.Sequential(
            nn.Linear(128 * 28 * 28, 256),  # adjust based on image size
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 2)  # 2 classes: chart / nonchart
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)  # flatten
        x = self.fc_layers(x)
        return x


# ----------------------------
# 2. Data Loading
# ----------------------------
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

train_dataset = datasets.ImageFolder(root="dataset/train", transform=transform)
val_dataset = datasets.ImageFolder(root="dataset/val", transform=transform)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# ----------------------------
# 3. Training Setup
# ----------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ChartClassifier().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# ----------------------------
# 4. Training Loop
# ----------------------------
epochs = 10
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}")

    # Validation
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f"Validation Accuracy: {100 * correct / total:.2f}%")

# ----------------------------
# 5. Save Model
# ----------------------------
os.makedirs("models", exist_ok=True)
torch.save(model.state_dict(), "models/chartdetector.pth")
print("Model saved to models/chartdetector.pth")


Epoch 1/10, Loss: 1.0662
Validation Accuracy: 63.64%
Epoch 2/10, Loss: 1.8862
Validation Accuracy: 63.64%
Epoch 3/10, Loss: 0.7216
Validation Accuracy: 36.36%
Epoch 4/10, Loss: 0.6897
Validation Accuracy: 36.36%
Epoch 5/10, Loss: 0.7399
Validation Accuracy: 63.64%
Epoch 6/10, Loss: 0.7040
Validation Accuracy: 63.64%
Epoch 7/10, Loss: 0.6924
Validation Accuracy: 63.64%
Epoch 8/10, Loss: 0.6938
Validation Accuracy: 63.64%
Epoch 9/10, Loss: 0.6845
Validation Accuracy: 63.64%
Epoch 10/10, Loss: 0.6935
Validation Accuracy: 63.64%
Model saved to models/chartdetector.pth
